## Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import tiktoken
import math
import re
import string

## Load Dataframe

In [2]:
# Load dataframe from `data` directory
merged_data = pd.read_parquet("../data/merged_data.parquet")

display(merged_data)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_images,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[[chas blue card, https://ch-api.healthhub.sg/...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,"b'<div class=""ExternalClassFC126593610D4F0587A...

In [3]:
df_keep = merged_data[~merged_data["to_remove"]]

display(df_keep)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_images,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[[chas blue card, https://ch-api.healthhub.sg/...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,"b'<div class=""ExternalClassE9019B25CD2A40F3948...",...,False,None,False,False,[],No

In [4]:
relevant_categories = [
    "cost-and-financing",
    "live-healthy-articles",
    "diseases-and-conditions",
    "medical-care-and-facilities",
    "support-group-and-others",
]

df_keep = df_keep[df_keep["content_category"].isin(relevant_categories)]

display(df_keep)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_images,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[[chas blue card, https://ch-api.healthhub.sg/...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,"b'<div class=""ExternalClassE9019B25CD2A40F3948...",...,False,None,False,False,[],No

In [5]:
df_extracted = df_keep[
    [
        "id",
        "content_name",
        "title",
        "article_category_names",
        "full_url",
        "friendly_url",
        "category_description",
        "content_category",
        "content_body",
        "pr_name",
        # "has_table",
        # "has_image",
        # "related_sections",
        # "extracted_tables",
        # "extracted_links",
        # "extracted_headers",
        # "extracted_img_alt_text",
        "extracted_content_body",
    ]
]

display(df_extracted)

,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,medisave,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,chas,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore","NCIS Breast Support Group provides knowledge, ..."
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",Heart failure is the leading cause of rehospit...
2609,1440791,Brain and Head Injury Support Groups,Brain and Head Injury Support Groups,"Alerts & Advisories,",https://www.healthhub.sg/a-z/support-groups-an...,2015-NNI-support-group,Read on for a list of brain injury support gro...,support-group-and-others,"b'<div class=""ExternalClass7C92735B78174928B28...",National Neuroscience Institute,Brain Tumour Society (Singapore)\nThe Brain Tu...
2610,1440768,Ambulatory Nutrition Support,Ambulatory Nutrition Support,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,ambulatory-nutrition-support,Read about the ambulatory support benefits one...,support-group-and-others,"b'<div class=""ExternalClass3FABAC9D59A64BCAB96...",National University Hospital,The Importance of Ambulatory Nutrition Support...


## Calculate total number of Tokens using `tiktoken`

In [6]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def calculate_word_count(text):
    sentences = []
    words = []

    # Compile regex to detect punctuations
    regex = re.compile("[%s]" % re.escape(string.punctuation))

    # Remove all hyperlinks
    filtered_text = re.sub(r"https?:\/\/[^\s]+", "", text)

    # Split the extracted text by the newline delimiter
    lines = filtered_text.split("\n")
    # Track the sentences in the text
    for line in lines:
        partial_sentences = re.split(r"[.!?]", line)
        for sentence in partial_sentences:
            sentences.append(sentence.strip())

    # Track the words in the text
    for sentence in sentences:
        sentence_words = sentence.split(" ")
        for word in sentence_words:
            word = regex.sub("", word.strip())
            words.append(word)

    # Filter for empty strings
    filtered_words = list(filter(lambda x: len(x) > 0, words))
    
    # Count the number of words
    num_words = len(filtered_words)
    return num_words

df_extracted.loc[:, "num_content_tokens"] = df_extracted["extracted_content_body"].apply(
    lambda x: num_tokens_from_string(x)
)

df_extracted.loc[:, "word_count"] = df_extracted["extracted_content_body"].apply(
    lambda x: calculate_word_count(x)
)

display(df_extracted.sort_values(by=["num_content_tokens"], ascending=False))

/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/3149991123.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted.loc[:, "num_content_tokens"] = df_extracted["extracted_content_body"].apply(
/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/3149991123.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted.loc[:, "word_count"] = df_extracted["extracted_content_body"].apply(


,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body,num_content_tokens,word_count
677,1446078,Pregnancy Complications,Pregnancy Complications,None,https://www.healthhub.sg/live-healthy/pregnanc...,pregnancy-complications,Complications range from vaginal bleeding to d...,live-healthy-articles,"b'<div class=""ExternalClass923270452D424E7986C...",KK Women's and Children's Hospital,There are several medical conditions that can ...,5025,3900
377,1446007,Do’s and Don’ts in Pregnancy,Do’s and Don’ts in Pregnancy,None,https://www.healthhub.sg/live-healthy/pregnanc...,pregnancy-dos-and-donts-in-pregnancy,Consuming folic acid capsules is one of the im...,live-healthy-articles,"b'<div class=""ExternalClass98DEB22FF048456F970...",KK Women's and Children's Hospital,"Food, Smoking, Alcohol and Recreational Drugs ...",4223,3435
372,1446016,Nutrition During Pregnancy—Eating Right for Two,Nutrition During Pregnancy—Eating Right for Two,None,https://www.healthhub.sg/live-healthy/pregnanc...,pregnancy-nutrition-during-pregnancy-eating-ri...,Can you have a healthy weight gain during preg...,live-healthy-articles,"b'<div class=""ExternalClass40DD4AB9D30E47EE98E...",KK Women's and Children's Hospital,"Source: Dr TAN Thiam Chye, Dr TAN Kim Teng, Dr...",4131,3133
1089,1444275,Guide 6: Pregnancy Issues Some Women Might Face,Guide 6: Pregnancy Issues Some Women Might Face,"Body Care,",https://www.healthhub.sg/live-healthy/guide-6-...,guide-6-pregnancy-issues-some-women-might-face,While pregnancy and delivery tend to be smooth...,live-healthy-articles,"b'<div class=""ExternalClass0F430CE9E53E409AA6E...",KK Women's and Children's Hospital,"By Dr Michelle LIM, Associate Consultant and A...",3678,2789
388,1442691,Feeding your baby: Breastfeeding,Feeding your baby: Breastfeeding,None,https://www.healthhub.sg/live-healthy/pregnanc...,pregnancy-feeding-your-baby-breastfeeding,It is never too early to start thinking about ...,live-healthy-articles,"b'<div class=""ExternalClassAE66C2352A7A4C6FB0B...",Health Promotion Board,Breastfeeding Positions and Concerns\nEvery pa...,3509,2817
...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,1445021,Baby’s First Year: Keep Your Eyes on Baby’s Vi...,Baby’s First Year: Keep Your Eyes on Baby’s Vi...,"Pregnancy and Infant Health,",https://www.healthhub.sg/live-healthy/baby-fir...,baby-first-year-keep-eyes-on-vision,"Want your baby to have healthy, bright eyes? H...",live-healthy-articles,"b'<div class=""ExternalClass207D628440EA4AB5952...",Health Promotion Board,Read these next:\n- Stimulate Your Baby's Firs...,23,16
1355,1445002,Child’s Sixth Year: Bite-sized Reminders for M...,Child’s Sixth Year: Bite-sized Reminders for M...,"Child and Teen Health,",https://www.healthhub.sg/live-healthy/child-si...,child-sixth-year-bite-sized-reminders,Quick reminders and tips to help care for your...,live-healthy-articles,"b'<div class=""ExternalClass8201B08542D34FF59C0...",Health Promotion Board,Read these next:\n- Health Screening for Prima...,22,15
968,1445019,Baby’s First Year: Tooth Fairy’s To-Do List,Baby’s First Year: Tooth Fairy’s To-Do List,"Pregnancy and Infant Health,",https://www.healthhub.sg/live-healthy/baby-fir...,baby-first-year-tooth-fairy-to-do-list,Here are some things you can do to take care o...,live-healthy-articles,"b'<div class=""ExternalClass996411A420844ED3A26...",Health Promotion Board,Read these next:\n- Help Your Baby Develop\n- ...,22,15
1360,1444991,Baby’s Second Year: Bite-sized Reminders for M...,Baby’s Second Year: Bite-sized Reminders for M...,"Child and Teen Health,",https://www.healthhub.sg/live-healthy/baby-sec...,baby-second-year-bite-sized-reminders,Quick reminders and tips to help care for your...,live-healthy-articles,"b'<div class=""ExternalClassCDA686B9D74344E2B33...",Health Promotion Board,Read these next:\n- Your Baby Needs Soft Skill...,19,15


In [7]:
total_tokens_across_articles = df_extracted["num_content_tokens"].sum()
print(f"Total Tokens across articles: {total_tokens_across_articles}")

Total Tokens across articles: 1310231


In [ ]:
fig = px.histogram(df_extracted, x="num_content_tokens", nbins=80)
fig.update_layout(
    title_text="Token Count distribution for Extracted Article Content",
    xaxis_title_text="Token Count",
    yaxis_title_text="Count",
    bargap=0.1,
)
# fig.add_vline(x=lower_threshold, line_dash="dash", line_color="firebrick")
# fig.add_vline(x=upper_threshold, line_dash="dash", line_color="firebrick")
fig.show()

In [9]:
print(
    df_extracted["num_content_tokens"].quantile(
        [0.001, 0.021, 0.136, 0.5, 0.841, 0.977, 0.999]
    )
)

0.001      20.524
0.021     142.668
0.136     326.088
0.500     754.000
0.841    1342.228
0.977    2325.896
0.999    4176.264
Name: num_content_tokens, dtype: float64


In [10]:
df_extracted.loc[:, "log_num_tokens"] = np.log(df_extracted["num_content_tokens"])

/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/1696151854.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
fig = px.histogram(df_extracted, x="log_num_tokens", nbins=80)
fig.update_layout(
    title_text="Log Token Count distribution for Extracted Article Content",
    xaxis_title_text="Token Count",
    yaxis_title_text="Count",
    bargap=0.1,
)
# fig.add_vline(x=lower_threshold, line_dash="dash", line_color="firebrick")
# fig.add_vline(x=upper_threshold, line_dash="dash", line_color="firebrick")
fig.show()

In [12]:
fig = px.box(df_extracted, x="log_num_tokens", color="content_category")
fig.update_layout(
    title_text="Box Plot for Log Token Count for Extracted Article Content",
    xaxis_title_text="Log Token Count",
)
fig.show()

In [13]:
df_hpb = df_extracted[df_extracted["pr_name"] == "Health Promotion Board"]
display(df_hpb.sort_values(by=["num_content_tokens"], ascending=False))

,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body,num_content_tokens,word_count,log_num_tokens
388,1442691,Feeding your baby: Breastfeeding,Feeding your baby: Breastfeeding,None,https://www.healthhub.sg/live-healthy/pregnanc...,pregnancy-feeding-your-baby-breastfeeding,It is never too early to start thinking about ...,live-healthy-articles,"b'<div class=""ExternalClassAE66C2352A7A4C6FB0B...",Health Promotion Board,Breastfeeding Positions and Concerns\nEvery pa...,3509,2817,8.163086
379,1445263,Let’s Get Real About Vaping – 8 Reasons Why Yo...,Let’s Get Real About Vaping – 8 Reasons Why Yo...,None,https://www.healthhub.sg/live-healthy/lets-get...,lets-get-real-about-vaping,"Thinking that it is cool and safe, many youth...",live-healthy-articles,"b'<div class=""ExternalClass19C32D9A0AD441FF972...",Health Promotion Board,Electronic-cigarettes have many names and are ...,3344,2067,8.114923
538,1445402,Facts on Tobacco Smoking and Tobacco Products,Facts on Tobacco Smoking and Tobacco Products,"Body Care,",https://www.healthhub.sg/live-healthy/facts_to...,facts_tobacco_products,Tobacco products comprise more than just cigar...,live-healthy-articles,"b'<div class=""ExternalClassC25E2263C6274587966...",Health Promotion Board,Facts About Tobacco and Tobacco Products\nMost...,3307,2226,8.103797
777,1442481,8 Personal Hygiene Boo Boos You Don't Want To ...,8 Personal Hygiene Boo Boos You Don't Want To ...,"FIGHT and Travel Health,",https://www.healthhub.sg/live-healthy/8-person...,8-personal-hygiene-boo-boos-you-dont-wanna-make,"Spare yourself from diarrhoea, the flu, and ot...",live-healthy-articles,"b'<div class=""ExternalClassDBDF8EF2D7074FB1B70...",Health Promotion Board,1. Not Putting the Lid Down When you Flush\nMo...,3222,2133,8.077758
1385,1445247,30 Ways to Go from Uncle to Hunkle,30 Ways to Go from Uncle to Hunkle,"Exercise and Fitness,",https://www.healthhub.sg/live-healthy/mens-hea...,mens-health-uncle-to-hunkle-bro-bonding,30 Fun Ways for males over the age of 30 to ge...,live-healthy-articles,"b'<div class=""ExternalClassFB3B4883B9064D40B36...",Health Promotion Board,Redefine and Reinvent Guys Night Out\nAccordin...,3146,2539,8.053887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,1445021,Baby’s First Year: Keep Your Eyes on Baby’s Vi...,Baby’s First Year: Keep Your Eyes on Baby’s Vi...,"Pregnancy and Infant Health,",https://www.healthhub.sg/live-healthy/baby-fir...,baby-first-year-keep-eyes-on-vision,"Want your baby to have healthy, bright eyes? H...",live-healthy-articles,"b'<div class=""ExternalClass207D628440EA4AB5952...",Health Promotion Board,Read these next:\n- Stimulate Your Baby's Firs...,23,16,3.135494
968,1445019,Baby’s First Year: Tooth Fairy’s To-Do List,Baby’s First Year: Tooth Fairy’s To-Do List,"Pregnancy and Infant Health,",https://www.healthhub.sg/live-healthy/baby-fir...,baby-first-year-tooth-fairy-to-do-list,Here are some things you can do to take care o...,live-healthy-articles,"b'<div class=""ExternalClass996411A420844ED3A26...",Health Promotion Board,Read these next:\n- Help Your Baby Develop\n- ...,22,15,3.091042
1355,1445002,Child’s Sixth Year: Bite-sized Reminders for M...,Child’s Sixth Year: Bite-sized Reminders for M...,"Child and Teen Health,",https://www.healthhub.sg/live-healthy/child-si...,child-sixth-year-bite-sized-reminders,Quick reminders and tips to help care for your...,live-healthy-articles,"b'<div class=""ExternalClass8201B08542D34FF59C0...",Health Promotion Board,Read these next:\n- Health Screening for Prima...,22,15,3.091042
1360,1444991,Baby’s Second Year: Bite-sized Reminders for M...,Baby’s Second Year: Bite-sized Reminders for M...,"Child and Teen Health,",https://www.healthhub.sg/live-healthy/baby-sec...,baby-second-year-bite-sized-reminders,Quick reminders and tips to help care for your...,live-healthy-articles,"b'<div class=""ExternalClassCDA686B9D74344E2B33..

In [14]:
fig = px.histogram(df_hpb, x="num_content_tokens", nbins=80)
fig.update_layout(
    title_text="Token Count distribution for Extracted Article Content",
    xaxis_title_text="Token Count",
    yaxis_title_text="Count",
    bargap=0.1,
)
# fig.add_vline(x=lower_threshold, line_dash="dash", line_color="firebrick")
# fig.add_vline(x=upper_threshold, line_dash="dash", line_color="firebrick")
fig.show()

In [15]:
fig = px.histogram(df_hpb, x="log_num_tokens", nbins=80)
fig.update_layout(
    title_text="Token Count distribution for Extracted Article Content",
    xaxis_title_text="Token Count",
    yaxis_title_text="Count",
    bargap=0.1,
)
# fig.add_vline(x=lower_threshold, line_dash="dash", line_color="firebrick")
# fig.add_vline(x=upper_threshold, line_dash="dash", line_color="firebrick")
fig.show()

In [16]:
fig = px.box(df_hpb, x="log_num_tokens", color="content_category")
fig.update_layout(
    title_text="Box Plot for Log Token Count for Extracted Article Content",
    xaxis_title_text="Log Token Count",
)
fig.show()

In [17]:
fig = px.scatter(df_hpb, x="num_content_tokens", y="word_count", color="content_category", trendline="ols", trendline_scope="overall")
fig.update_layout(
    title_text="Scatterplot of Token Count with respect to Word Count",
    xaxis_title_text="Token Count",
    yaxis_title_text=" Word Count",
)
fig.show()

In [18]:
results = px.get_trendline_results(fig)
print(results)

results.px_fit_results.iloc[0].summary()

                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                 1.903e+04
Date:                Fri, 26 Jul 2024   Prob (F-statistic):               0.00
Time:                        09:01:06   Log-Likelihood:                -3856.5
No. Observations:                 668   AIC:                             7717.
Df Residuals:                     666   BIC:                             7726.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.7218      6.212      4.141      0.000      13.525      37.919
x1             0.7393      0.005    137.950      0.000       0.729       0.750
==============================================================================
Omnibus:                      360.322   Durbin-Watson:                   1.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4300.043
Skew:                          -2.141   Prob(JB):                         0.00
Kurtosis:                      14.669   Cond. No.                     2.39e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Based on the statistics above, 1 token ~= 0.75 words. We will use this measure going forward

## Expected Cost Calculation

### Cost of OpenAI models

In [19]:
## Costs per 1000 tokens
models_costs = {
    "gpt-3.5-turbo-0125": {"inputs": 0.0005, "outputs": 0.0015},
    "gpt-4o": {"inputs": 0.005, "outputs": 0.015},
    "gpt-4o-mini": {"inputs": 0.15 / 1000, "outputs": 0.60 / 1000},
}

print(models_costs)

{'gpt-3.5-turbo-0125': {'inputs': 0.0005, 'outputs': 0.0015}, 'gpt-4o': {'inputs': 0.005, 'outputs': 0.015}, 'gpt-4o-mini': {'inputs': 0.00015, 'outputs': 0.0006}}


#### Number of Agents, Max Number of Similar Articles, Percentage of articles to optimise/harmonise

In [20]:
## Optimisation
# Meta Desc Quality Eval, Title Quality Eval, Content Quality Eval, Researcher, Meta Desc Optimiser, Title Optimiser, Content Guidelines, Writing Guidelines
# Percentage of articles to optimise
percentage_to_optimise = 0.4

## Harmonisation
# Researcher, Compiler, Meta Desc Optimiser, Title Optimiser, Content Guidelines, Writing Guidelines
# Number of similar articles (max. 5)
similar_articles_count = 5

# Percentage of articles to combine/harmonise
percentage_to_harmonise = 0.3

In [21]:
# Generated evaluations
generated_evaluation_tokens = 200 # Approx. 150 words for generated text from evaluating meta desc, title and content quality each respectively

# Title and Meta Desc tokens
title_tokens = 25 # Approx. 70 characters / 16 words
meta_desc_tokens = 75  # Approx 50 words

# Generated Output Multiplier (with respect to article tokens)
keypoints_generated_multiplier = 1.1
compiled_articles_multiplier = 0.8 # An estimate of the generated article as compared to compiled articles 
content_guidelines_multiplier = 1.25
writing_guidelines_multiplier = content_guidelines_multiplier * 1.25

## Prompt Tokens
evaluation_prompt_tokens = 200  # Approx. 150 words for evaluating meta desc, title and content quality respectively
researcher_prompt_tokens = 400  # Approx. 300 words for instructing researcher
compiler_prompt_tokens = 400    # Approx. 300 words for instructing compiler
guidelines_prompt_tokens = 600  # Approx. 450 words for guidelines
meta_desc_prompt_tokens = 200   # Approx. 150 words for prompt instructions
title_prompt_tokens = 200       # Approx. 150 words for prompt instructions


# USD to SGD Conversion
usd_to_sgd = 1.35

tokens_dict = {
    "percentage_to_optimise": percentage_to_optimise,
    "similar_articles_count": similar_articles_count,
    "percentage_to_harmonise": percentage_to_harmonise,
    "title_tokens": title_tokens,
    "meta_desc_tokens": meta_desc_tokens,
    "keypoints_generated_multiplier": keypoints_generated_multiplier,
    "compiled_articles_multiplier": compiled_articles_multiplier,
    "content_guidelines_multiplier": content_guidelines_multiplier,
    "writing_guidelines_multiplier": writing_guidelines_multiplier,
    "evaluation_prompt_tokens": evaluation_prompt_tokens,
    "researcher_prompt_tokens": researcher_prompt_tokens,
    "compiler_prompt_tokens": compiler_prompt_tokens,
    "guidelines_prompt_tokens": guidelines_prompt_tokens,
    "meta_desc_prompt_tokens": meta_desc_prompt_tokens,
    "title_prompt_tokens": title_prompt_tokens,
    "usd_to_sgd": usd_to_sgd,
}

In [22]:
def calculate_tokens_optimisation(article_tokens: int, params: dict[str, int] = tokens_dict) -> int:
    # Prompt Tokens
    evaluation_prompt_tokens = params.get("evaluation_prompt_tokens", 200)
    researcher_prompt_tokens = params.get("researcher_prompt_tokens", 400)
    guidelines_prompt_tokens = params.get("guidelines_prompt_tokens", 600)
    meta_desc_prompt_tokens = params.get("meta_desc_prompt_tokens", 200)
    title_prompt_tokens = params.get("title_prompt_tokens", 200)
    
    # Generated Tokens
    title_tokens = params.get("title_tokens", 25)
    meta_desc_tokens = params.get("meta_desc_tokens", 75)
    generated_evaluation_tokens = params.get("generated_evaluation_tokens", 200)
    
    # Multipliers
    keypoints_generated_multiplier = params.get("keypoints_generated_multiplier", 0.5)
    content_guidelines_multiplier = params.get("content_guidelines_multiplier", 1.25)
    writing_guidelines_multiplier = params.get("writing_guidelines_multiplier", 1.5)
    
    # Meta Desc, Title and Content Quality Evaluation - Token Calculation
    evaluation_input_tokens = evaluation_prompt_tokens * 3 + article_tokens + title_tokens + meta_desc_tokens
    evaluation_output_tokens = generated_evaluation_tokens * 3
    
    # Researcher -> Keypoints - Token Calculation
    researcher_input_tokens = researcher_prompt_tokens + article_tokens
    researcher_output_tokens = article_tokens * keypoints_generated_multiplier
    
    # Content Guidelines - Token Calculation
    content_guidelines_input_tokens = guidelines_prompt_tokens + researcher_output_tokens
    content_guidelines_output_tokens = article_tokens * content_guidelines_multiplier
    
    # Writing Guidelines - Token Calculation
    writing_guidelines_input_tokens = guidelines_prompt_tokens + content_guidelines_output_tokens
    writing_guidelines_output_tokens = article_tokens * writing_guidelines_multiplier
    
    # Title Optimisation - Token Calculation
    title_optimisation_input_tokens = title_prompt_tokens + writing_guidelines_output_tokens
    title_optimisation_output_tokens = title_tokens
    
    # Meta Desc Optimisation - Token Calculation
    meta_desc_input_tokens = meta_desc_prompt_tokens + writing_guidelines_output_tokens
    meta_desc_output_tokens = meta_desc_tokens
    
    # Total Tokens Calculation
    total_input_tokens = math.ceil(evaluation_input_tokens + researcher_input_tokens + content_guidelines_input_tokens + writing_guidelines_input_tokens + title_optimisation_input_tokens + meta_desc_input_tokens)
    total_output_tokens = math.ceil(evaluation_output_tokens + researcher_output_tokens + content_guidelines_output_tokens + writing_guidelines_output_tokens + title_optimisation_output_tokens + meta_desc_output_tokens)
    
    return total_input_tokens, total_output_tokens

In [23]:
def calculate_optimisation_costs(article_tokens: int, model: str = "gpt-3.5-turbo-0125", params: dict[str, int] = tokens_dict) -> int:
    usd_to_sgd = params.get("usd_to_sgd", 1.35)
    
    inputs, outputs = calculate_tokens_optimisation(article_tokens, params)
    optimisation_cost = (inputs * models_costs[model]["inputs"] + outputs * models_costs[model]["outputs"]) / 1000 * usd_to_sgd
    
    return optimisation_cost

In [24]:
model = "gpt-3.5-turbo-0125"
col = f"optimise_costs_{model}"
df_extracted[col] = df_extracted["num_content_tokens"].apply(lambda x: calculate_optimisation_costs(x, model, tokens_dict))

display(df_extracted)

/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/3410570429.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body,num_content_tokens,word_count,log_num_tokens,optimise_costs_gpt-3.5-turbo-0125
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,Breast cancer is the number one cancer among w...,764,563,6.638568,0.013150
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,MediSave Maternity Package\nWith the MediSave ...,1171,873,7.065613,0.018428
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,medisave,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,"What is MediSave?\nMediSave, introduced in Apr...",491,366,6.196444,0.009610
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,Patients or family members who have difficulty...,374,324,5.924256,0.008092
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,chas,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,What is the Community Health Assist Scheme (CH...,752,567,6.622736,0.012994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore","NCIS Breast Support Group provides knowledge, ...",281,224,5.638355,0.006886
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",Heart failure is the leading cause of rehospit...,533,398,6.278521,0.010154
2609,1440791,Brain and Head Injury Support Groups,Brain and Head Injury Support Groups,"Alerts & Advisories,",https://www.healthhub.sg/a-z/support-groups-an...,2015-NNI-support-group,Read on for a list of brain injury support gro...,support-group-and-others,"b'<div class=""ExternalClass7C92735B78174928B28...",National Neuroscience Institute,Brain Tumour Society (Singapore)\nThe Brain Tu...,945,542,6.851185,0.015497
2610,1440768,Ambulatory Nutrition Support,Ambulatory Nutrition Support,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,ambulatory-nutrition-support,Read about the ambulatory support benefits one...,support-group-and-others,"b'<div class=""ExternalClass3FABAC9D59A64BCAB96...",National University Hospital,The Importance of Ambulatory Nutrition Support...,440,317,6.086775,0.00894

In [25]:
model = "gpt-4o-mini"
col = f"optimise_costs_{model}"
df_extracted[col] = df_extracted["num_content_tokens"].apply(lambda x: calculate_optimisation_costs(x, model, tokens_dict))

display(df_extracted)

/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/3885543187.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body,num_content_tokens,word_count,log_num_tokens,optimise_costs_gpt-3.5-turbo-0125,optimise_costs_gpt-4o-mini
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,Breast cancer is the number one cancer among w...,764,563,6.638568,0.013150,0.004692
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,MediSave Maternity Package\nWith the MediSave ...,1171,873,7.065613,0.018428,0.006598
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,medisave,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,"What is MediSave?\nMediSave, introduced in Apr...",491,366,6.196444,0.009610,0.003414
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,Patients or family members who have difficulty...,374,324,5.924256,0.008092,0.002866
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,chas,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,What is the Community Health Assist Scheme (CH...,752,567,6.622736,0.012994,0.004636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore","NCIS Breast Support Group provides knowledge, ...",281,224,5.638355,0.006886,0.002430
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",Heart failure is the leading cause of rehospit...,533,398,6.278521,0.010154,0.003610
2609,1440791,Brain and Head Injury Support Groups,Brain and Head Injury Support Groups,"Alerts & Advisories,",https://www.healthhub.sg/a-z/support-groups-an...,2015-NNI-support-group,Read on for a list of brain injury support gro...,support-group-and-others,"b'<div class=""ExternalClass7C92735B78174928B28...",National Neuroscience Institute,Brain Tumour Society (Singapore)\nThe Brain Tu...,945,542,6.851185,0.015497,0.005540
2610,1440768,Ambulatory Nutrition Support,Ambulatory Nutrition Support,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,ambulatory-nutrition-support,Read about the ambulatory support benefits one...,support-group-and-others,"b'<div class=""ExternalClass3FABAC9D59A64BCAB96...",

In [26]:
model = "gpt-4o"
col = f"optimise_costs_{model}"
df_extracted[col] = df_extracted["num_content_tokens"].apply(lambda x: calculate_optimisation_costs(x, model, tokens_dict))

display(df_extracted)

/var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_23162/2599079183.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,content_name,title,article_category_names,full_url,friendly_url,category_description,content_category,content_body,pr_name,extracted_content_body,num_content_tokens,word_count,log_num_tokens,optimise_costs_gpt-3.5-turbo-0125,optimise_costs_gpt-4o-mini,optimise_costs_gpt-4o
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,Breast cancer is the number one cancer among w...,764,563,6.638568,0.013150,0.004692,0.131497
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,MediSave Maternity Package\nWith the MediSave ...,1171,873,7.065613,0.018428,0.006598,0.184275
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,medisave,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,"What is MediSave?\nMediSave, introduced in Apr...",491,366,6.196444,0.009610,0.003414,0.096100
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,Patients or family members who have difficulty...,374,324,5.924256,0.008092,0.002866,0.080919
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,chas,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,What is the Community Health Assist Scheme (CH...,752,567,6.622736,0.012994,0.004636,0.129944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore","NCIS Breast Support Group provides knowledge, ...",281,224,5.638355,0.006886,0.002430,0.068857
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",Heart failure is the leading cause of rehospit...,533,398,6.278521,0.010154,0.003610,0.101540
2609,1440791,Brain and Head Injury Support Groups,Brain and Head Injury Support Groups,"Alerts & Advisories,",https://www.healthhub.sg/a-z/support-groups-an...,2015-NNI-support-group,Read on for a list of brain injury support gro...,support-group-and-others,"b'<div class=""ExternalClass7C92735B78174928B28...",National Neuroscience Institute,Brain Tumour Society (Singapore)\nThe Brain Tu...,945,542,6.851185,0.015497,0.005540,0.154967
2610,1440768,Ambulatory Nutrition Support,Ambulatory Nutrition Support,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,ambulatory-nutrition-support,Read about the ambulatory supp

In [27]:
df_extracted.to_excel(f"../data/articles_optimisation_costs.xlsx", index=False)

In [28]:
def calculate_tokens_harmonisation(article_tokens_list, model: str = "gpt-3.5-turbo", params: dict[str, int] = tokens_dict) -> int:
    # Prompt Tokens
    researcher_prompt_tokens = params.get("researcher_prompt_tokens", 400)
    compiler_prompt_tokens = params.get("compiler_prompt_tokens", 400)
    guidelines_prompt_tokens = params.get("guidelines_prompt_tokens", 600)
    meta_desc_prompt_tokens = params.get("meta_desc_prompt_tokens", 200)
    title_prompt_tokens = params.get("title_prompt_tokens", 200)
    
    # Generated Tokens
    title_tokens = params.get("title_tokens", 25)
    meta_desc_tokens = params.get("meta_desc_tokens", 75)
    
    # Multipliers
    keypoints_generated_multiplier = params.get("keypoints_generated_multiplier", 0.5)
    compiled_articles_multiplier = params.get("compiled_articles_multiplier", 0.8)
    content_guidelines_multiplier = params.get("content_guidelines_multiplier", 1.25)
    writing_guidelines_multiplier = params.get("writing_guidelines_multiplier", 1.5)
    
    researcher_input_tokens = 0
    researcher_output_tokens = 0
    
    # Researcher -> Keypoints
    for i in range(len(article_tokens_list)):
        researcher_input_tokens += researcher_prompt_tokens + article_tokens_list[i]
        researcher_output_tokens += article_tokens_list[i] * keypoints_generated_multiplier
    
    # Compiler
    compiler_input_tokens = compiler_prompt_tokens + researcher_output_tokens
    compiler_output_tokens = researcher_output_tokens
    
    # Content Guidelines
    content_guidelines_input_tokens = guidelines_prompt_tokens + compiler_output_tokens
    content_guidelines_output_tokens = sum(article_tokens_list) * compiled_articles_multiplier * content_guidelines_multiplier
    
    # Writing Guidelines
    writing_guidelines_input_tokens = guidelines_prompt_tokens + content_guidelines_output_tokens
    writing_guidelines_output_tokens = sum(article_tokens_list) * compiled_articles_multiplier * writing_guidelines_multiplier
    
    # Title Optimisation
    title_optimisation_input_tokens = title_prompt_tokens + writing_guidelines_output_tokens
    title_optimisation_output_tokens = title_tokens
    
    # Meta Desc Optimisation
    meta_desc_input_tokens = meta_desc_prompt_tokens + writing_guidelines_output_tokens
    meta_desc_output_tokens = meta_desc_tokens
    
    # Total
    total_input_tokens = math.ceil(researcher_input_tokens + compiler_input_tokens + content_guidelines_input_tokens + writing_guidelines_input_tokens + title_optimisation_input_tokens + meta_desc_input_tokens)
    total_output_tokens = math.ceil(researcher_output_tokens + compiler_output_tokens + content_guidelines_output_tokens + writing_guidelines_output_tokens + title_optimisation_output_tokens + meta_desc_output_tokens)
    return total_input_tokens, total_output_tokens

In [29]:
def calculate_costs(tokens_array, model, params: dict[str, int] = tokens_dict):
    usd_to_sgd = params.get("usd_to_sgd", 1.35)
    
    rng = np.random.default_rng()
    rng.shuffle(tokens_array)
    # print(tokens_array)
    
    optimization_size = round(percentage_to_optimise * tokens_array.size)
    # print(optimization_size)
    
    harmonisation_size = int((percentage_to_harmonise * tokens_array.size // similar_articles_count + 1) * similar_articles_count)
    # print(harmonisation_size)
    
    optimization_tokens_array = tokens_array[:optimization_size]
    harmonisation_tokens_array = tokens_array[optimization_size:optimization_size+harmonisation_size].reshape(-1, similar_articles_count)
    # print(len(harmonisation_tokens_array))
    
    optimisation_cost = 0
    for i in range(optimization_tokens_array.shape[0]):
        inputs, outputs = calculate_tokens_optimisation(optimization_tokens_array[i])
        optimisation_cost += (inputs * models_costs[model]["inputs"] + outputs * models_costs[model]["outputs"]) / 1000 * usd_to_sgd    # Return cost in SGD
    
    print(f"Optimisation cost: {optimisation_cost}")
    
    harmonisation_cost = 0
    for i in range(harmonisation_tokens_array.shape[0]):
        inputs, outputs = calculate_tokens_harmonisation(harmonisation_tokens_array[i])
        harmonisation_cost += (inputs * models_costs[model]["inputs"] + outputs * models_costs[model]["outputs"]) / 1000 * usd_to_sgd   # Return cost in SGD
    
    print(f"Harmonisation cost: {harmonisation_cost}")
    
    cost = optimisation_cost + harmonisation_cost
    
    return cost  

In [30]:
def calculate_average_costs(tokens_array, model, runs = 30):
    costs = []
    for i in range(runs):
        cost = calculate_costs(tokens_array, model)
        costs.append(cost)
    
    avg_cost = sum(costs) / runs
    print(tokens_array.shape[0])
    print(f"Average cost: {avg_cost}")
    avg_cost_per_article = avg_cost / ((percentage_to_optimise + percentage_to_harmonise) * tokens_array.shape[0])
    print(f"Average cost Per article: {avg_cost_per_article}")
    
    return avg_cost

### All articles

In [31]:
tokens_array = np.copy(df_extracted["num_content_tokens"].to_numpy())
print(tokens_array)

[ 764 1171  491 ...  945  440  171]


In [32]:
calculate_costs(tokens_array, model="gpt-4o-mini")

Optimisation cost: 3.161180114999996
Harmonisation cost: 2.015908965


5.1770890799999965

In [41]:
calculate_average_costs(tokens_array, model="gpt-3.5-turbo-0125") # gpt-3.5-turbo-0125, gpt-4o-mini, gpt-4o

Optimisation cost: 8.760535424999997
Harmonisation cost: 5.496574275000001
Optimisation cost: 8.473949325000003
Harmonisation cost: 5.901687000000001
Optimisation cost: 8.907538275000011
Harmonisation cost: 5.668620975000001
Optimisation cost: 8.743378275000001
Harmonisation cost: 5.52466575
Optimisation cost: 8.895512474999991
Harmonisation cost: 5.464314674999998
Optimisation cost: 8.716547025000002
Harmonisation cost: 5.691983400000001
Optimisation cost: 8.87674477499999
Harmonisation cost: 5.616249075000001
Optimisation cost: 8.683464599999999
Harmonisation cost: 5.576675850000001
Optimisation cost: 8.661284774999999
Harmonisation cost: 5.899779450000001
Optimisation cost: 9.06563812499999
Harmonisation cost: 5.535160649999999
Optimisation cost: 8.728913700000001
Harmonisation cost: 5.450926050000001
Optimisation cost: 8.484282899999998
Harmonisation cost: 5.825362725000001
Optimisation cost: 8.792937450000004
Harmonisation cost: 5.774440724999998
Optimisation cost: 8.5840505999999

14.363638199999999

### HPB Articles

In [34]:
hpb_tokens_array = np.copy(df_hpb["num_content_tokens"].to_numpy())
print(tokens_array)

[1041  764  762 ... 1128 2153  820]


In [35]:
calculate_costs(hpb_tokens_array, model="gpt-3.5-turbo-0125")

Optimisation cost: 4.314999599999998
Harmonisation cost: 3.008498625


7.323498224999998

In [40]:
calculate_average_costs(hpb_tokens_array, model="gpt-3.5-turbo-0125") # gpt-3.5-turbo-0125, gpt-4o-mini, gpt-4o

Optimisation cost: 4.423151474999996
Harmonisation cost: 2.8447672500000007
Optimisation cost: 4.421539575000001
Harmonisation cost: 2.919594375
Optimisation cost: 4.387666050000003
Harmonisation cost: 2.7955928250000004
Optimisation cost: 4.291290225000004
Harmonisation cost: 3.0167984250000006
Optimisation cost: 4.51318365
Harmonisation cost: 2.8717220249999995
Optimisation cost: 4.1918202
Harmonisation cost: 3.0001549499999993
Optimisation cost: 4.331912400000002
Harmonisation cost: 2.92997385
Optimisation cost: 4.428880199999999
Harmonisation cost: 3.0707896500000005
Optimisation cost: 4.405600125000002
Harmonisation cost: 2.9406010499999997
Optimisation cost: 4.326947100000001
Harmonisation cost: 2.911605075
Optimisation cost: 4.364282700000001
Harmonisation cost: 2.9239089750000007
Optimisation cost: 4.361259375
Harmonisation cost: 2.8956966750000004
Optimisation cost: 4.403302425000002
Harmonisation cost: 3.104948025
Optimisation cost: 4.446079199999999
Harmonisation cost: 2.799

7.3168467525